In [1]:
import pandas as pd

In [2]:
#importing data
data_su = pd.read_csv("Suicide_Detection.csv")
data_su.drop("Unnamed: 0",inplace=True,axis=1)
data_su.head(10)

,text,class
0,Ex Wife Threatening SuicideRecently I left my ...,suicide
1,Am I weird I don't get affected by compliments...,non-suicide
2,Finally 2020 is almost over... So I can never ...,non-suicide
3,i need helpjust help me im crying so hard,suicide
4,"I’m so lostHello, my name is Adam (16) and I’v...",suicide
5,Honetly idkI dont know what im even doing here...,suicide
6,[Trigger warning] Excuse for self inflicted bu...,suicide
7,It ends tonight.I can’t do it anymore. \nI quit.,suicide
8,"Everyone wants to be ""edgy"" and it's making me...",non-suicide
9,My life is over at 20 years oldHello all. I am...,suicide


In [3]:
data_su.shape

(232074, 2)

In [4]:
import re
import spacy
import numpy as np
import pandas as pd
import en_core_web_sm
import tensorflow as tf
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from spacy.lang.en.stop_words import STOP_WORDS
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
nlp = en_core_web_sm.load()
lemmatizer = WordNetLemmatizer()
stopwords = STOP_WORDS
EMOJI_PATTERN = re.compile(
    "["
    u"U0001F600-U0001F64F"  # emoticons
    u"U0001F300-U0001F5FF"  # symbols & pictographs
    u"U0001F680-U0001F6FF"  # transport & map symbols
    u"U0001F1E0-U0001F1FF"  # flags (iOS)
    u"U00002702-U000027B0"
    u"U000024C2-U0001F251"
    "]+", flags=re.UNICODE
)
FILTERS = '!"#$%&()*+,-/:;?@[\]^_`{|}~tn'
HTML_TAG_PATTERN = re.compile(r']*>')
NUMBERING_PATTERN = re.compile('d+(?:st|[nr]d|th)')
DISABLE_PIPELINES = ["tok2vec", "parser", "ner", "textcat", "custom", "lemmatizer"]

2023-08-11 18:50:41.231307: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-11 18:50:41.925324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
def initial_preprocessing(text):
    tag_removed_text = HTML_TAG_PATTERN.sub('', text)
    emoji_removed_text = EMOJI_PATTERN.sub(r'', tag_removed_text)
    numberings_removed_text =  NUMBERING_PATTERN.sub('', emoji_removed_text)

    extra_chars_removed_text = re.sub(

        r"(.)1{2,}",  r'11', numberings_removed_text

    )

    return extra_chars_removed_text



In [6]:

def preprocess_text(doc):
    tokens = [

        token

        for token in doc

        if not token.is_space and

           not token.like_email and

           not token.like_url and

           not token.is_stop and

           not token.is_punct and

           not token.like_num

    ]
    translation_table = str.maketrans('', '', FILTERS)

    translated_tokens = [

        token.text.lower().translate(translation_table)

        for token in tokens

    ]

    lemmatized_tokens = [

        lemmatizer.lemmatize(token)

        for token in translated_tokens

        if len(token) > 1

    ]

    return lemmatized_tokens



In [ ]:
data_su['class'].unique()

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
labels = data_su['class'].iloc[:10000]

labels = labels.map(lambda x: 1 if x=='non-suicide' else 0)

data = data_su.iloc[:10000, :]

column = 'text'

not_null_data = data[data[column].notnull()]

not_null_data[column] = not_null_data[column].apply(initial_preprocessing)

texts = [
    preprocess_text(doc)
    for doc in nlp.pipe(not_null_data[column], disable=DISABLE_PIPELINES)
]

In [ ]:
texts[0]

In [ ]:
tokenizer = Tokenizer(
    filters=FILTERS,
    lower=True
)
padding = 'post'
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = []
max_sequence_len = 0
for text in texts:
    # convert texts to sequence
    txt_to_seq = tokenizer.texts_to_sequences([text])[0]
    sequences.append(txt_to_seq)
    # find max_sequence_len for padding
    txt_to_seq_len = len(txt_to_seq)
    if txt_to_seq_len > max_sequence_len:
        max_sequence_len = txt_to_seq_len
# post padding
print(max_sequence_len)
padded_sequences = pad_sequences(
    sequences, 
    maxlen=max_sequence_len, 
    padding=padding
)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=max_sequence_len))
model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(None, 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
adam = Adam(learning_rate=0.01)
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    optimizer=adam, 
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(
    padded_sequences, 
    labels.values, 
    epochs=15,
    verbose=1,
    batch_size=64
)

In [ ]:
import matplotlib.pyplot as plt
fig = plt.plot(history.history['accuracy'])
title = plt.title("History")
xlabel = plt.xlabel("Epochs")
ylabel = plt.ylabel("Accuracy")

In [ ]:
predictions = pickled_model.predict(padded_sequences[:7])
for pred in predictions:
    print(pred[0])

In [ ]:
data_su.head(7)

In [24]:

def predict (model,txt):
    length = len(txt)
    ini_txt = list(map(initial_preprocessing,txt))

    texts = [
        preprocess_text(doc)
        for doc in nlp.pipe(ini_txt, disable=DISABLE_PIPELINES)
    ]
    tokenizer = Tokenizer(
        filters=FILTERS,
        lower=True
    )
    new_data_seq = tokenizer.texts_to_sequences(texts)
    
    # padding
    new_data_seq_padded = pad_sequences(new_data_seq, maxlen=1505)
    
    
    predictions = model.predict(new_data_seq_padded)
    print(predictions)

In [25]:
def return_mental_state(val):
    if val > 0.5:
        return("Not-Suicide")
    return ("Suicide")

In [26]:
#custom testing
txt = ["what am I living for", "I am so happy", "what happede to that guy was sad"]

preds = predict(pickled_model,txt)


1/1 [==============================] - 0s 89ms/step
[[0.7398981]
 [0.7398981]
 [0.7398981]]


In [18]:
import pickle

In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))

In [8]:
pickled_model = pickle.load(open('model.pkl', 'rb'))

2023-08-11 18:50:54.193447: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-11 18:50:54.194427: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
